In [ ]:
!pip install xlrd

In [ ]:
import xlrd
import openpyxl
C_ID=1;  C_NAME=2; C_DESC=3; C_TYPE=4; C_GOODBEFORE_VAL=11;  C_GOODBEFORE_GRAN=C_GOODBEFORE_VAL+1;
C_GOODAFTER_VAL=13 ; C_GOODAFTER_GRAN=C_GOODAFTER_VAL+1; C_NUM_MIN = 15; C_NUM_MAX = 16; C_UNITS=17;
C_NOM_ALLOWEDVALS = 18; C_WRAPPINGID=19;C_LOW = 20;C_NORMAL_LOW = 21;C_NORMAL_HIGH = 22;C_HIGH = 23

In [ ]:
def get(row, col):
    return wb.active.cell(row=row, column=col).value


In [ ]:
def generateXML_raw_concept(row):
    xml="<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n"
    xml+='<';

    id=int(get(row,C_ID));
    if (id==0):
        return "",""
    name=get(row,C_NAME); desc=get(row,C_DESC);  type=get(row,C_TYPE)
    goodBeforeValue=int(get(row,C_GOODBEFORE_VAL));    goodBeforeGranularity=get(row,C_GOODBEFORE_GRAN)
    goodAfterValue=int(get(row,C_GOODAFTER_VAL)) ;     goodAfterGranularity=get(row,C_GOODAFTER_GRAN)
    allowedValuesList=get(row,C_NOM_ALLOWEDVALS)
    try :
        allowedValuesList=allowedValuesList.split(',')
        strippedTemp =[]
        for val in allowedValuesList:
            val=val.strip()
            strippedTemp.append(val)
        allowedValuesList=strippedTemp
    except:
        pass

    if (type=='raw-numeric'):
        xml+= 'numeric-raw-concept '
        minVal=int(get(row,C_NUM_MIN)); maxVal=int(get(row,C_NUM_MAX))
        units=get(row,C_UNITS)

    elif (type=='raw-nominal'):
        xml+= "nominal-raw-concept "
        allowedVals=['a']
    else: xml+- '!MISSING TYPE!'
    if desc is None:
        desc = ''
    xml+='id=\"'+str(id)+"\" "+"name=\""+name+"\" \nconcept-type=\""+\
         type+"\" description=\""+desc+"\" semi-formal=\"String\" source=\"String\" "+\
         "xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:noNamespaceSchemaLocation=\"C:\\Users\\mashi\\data science\\thesis\\16.10.xsd\"> "+\
        "\n<categories/>"+\
        "\n<standard-terms/>"+\
        "\n<synonyms/>"+\
        "\n<temporal-semantic downward-hereditary=\"true\" forward=\"true\" backward=\"true\" solid=\"true\" concatenable=\"true\" gestalt=\"true\"/>"+\
        "\n<clippers/>\n"

    xmlPersistence="<persistence>"+\
                "\n<global-persistence behavior=\"pos-pos\" granularity=\"second\">"+\
                    "\n<interpolation-table>"+\
                        "\n<rows/>"+\
                "\n	</interpolation-table>"+\
                "\n</global-persistence>"+\
            "\n	<local-persistence>"+\
                "\n	<good-before value=\""+str(goodBeforeValue)+"\" granularity=\""+goodBeforeGranularity+"\"/>"+\
                "\n	<good-after value=\""+str(goodAfterValue)+"\" granularity=\""+goodAfterGranularity+"\"/>"+\
                "\n</local-persistence>"+\
            "\n</persistence>"

    if (type=='raw-numeric'):
        xml+="<numeric-allowed-values output-type=\"numeric\" min-value=\""+str(minVal)+"\" max-value=\""+str(maxVal)+"\" units=\""+units+"\" scale=\"ratio\">\n"
        xml+=xmlPersistence
        xml+="\n</numeric-allowed-values>"+\
        "\n</numeric-raw-concept>"
    elif (type=='raw-nominal'):
        xml+="<nominal-allowed-values output-type=\"nominal\">\n"
        xml+=xmlPersistence
        xml+="<values>\n"
        for v in allowedValuesList:
            xml+="<nominal-allowed-value value=\""+v+"\"/>\n"
        xml+="</values>\n"
        xml+="</nominal-allowed-values>"+\
         "\n</nominal-raw-concept>"

    return name,xml


In [ ]:
##need to fix id's
##check about xmlns:xs
def generateXML_state(row):
    xml="<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n"
    xml+='<';

    id=int(get(row,C_ID));##THIS NEED TO BE THE ID OF THE RAW CONCEPT
    derived_id =int(get(row,C_WRAPPINGID))
    if (id==0):
        return "",""
    name=get(row,C_NAME); desc=get(row,C_DESC);  type='state'
    goodBeforeValue=int(get(row,C_GOODBEFORE_VAL));    goodBeforeGranularity=get(row,C_GOODBEFORE_GRAN)
    goodAfterValue=int(get(row,C_GOODAFTER_VAL)) ;     goodAfterGranularity=get(row,C_GOODAFTER_GRAN)
    allowedValuesList=get(row,C_NOM_ALLOWEDVALS);low_val = get(row,C_LOW);normal_low_val = get(row,C_NORMAL_LOW);
    normal_high_val = get(row,C_NORMAL_HIGH);high_val = get(row,C_HIGH)
    try :
        allowedValuesList=allowedValuesList.split(',')
        strippedTemp =[]
        for val in allowedValuesList:
            val=val.strip()
            strippedTemp.append(val)
        allowedValuesList=strippedTemp
    except:
        pass

    units=get(row,C_UNITS)
    if desc is None:
        desc = ''
    ##the id here needs to be the id of the statw
    xml+='state id=\"'+str(id)+"\" "+"name=\""+name+'_STATE'"\" \nconcept-type=\""+\
         type+"\" description=\""+desc+"\" semi-formal=\"String\" source=\"String\" "+\
         "xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:noNamespaceSchemaLocation=\"C:\\Users\\mashi\\data science\\thesis\\16.10.xsd\"> "+\
        "\n	<categories/>"+\
        "\n	<standard-terms/>"+\
        "\n	<synonyms/>"+\
        "\n	<temporal-semantic downward-hereditary=\"true\" forward=\"true\" backward=\"true\" solid=\"true\" concatenable=\"true\" gestalt=\"true\"/>"+\
        "\n	<clippers/>"+\
        "\n	<derived-from>"+\
        "\n		<derived-from-id>"+str(derived_id)+"</derived-from-id>"+\
        "\n	</derived-from>"+\
        "\n	<abstraction-at-contexts/>"###what id to put here
    ##check about data type
    xml+= '\n	<mapping-function rank-selection-criteria="min">'+\
        '\n		<mapping-functions-to-values>'+\
        '\n			<mapping-function-2-value order="0" value="Low">'+\
        '\n				<evaluation-tree data-type="comparison-function">'+\
        '\n					<comparison-function comparison-operator="smaller">'+\
        '\n						<left data-type="concept-id-allowed-values">'+\
        '\n							<concept-id-allowed-values id="{}"/>'.format(str(derived_id))+\
        '\n						</left>'+\
        '\n						<right data-type="double">'+\
        '\n							<double>{}</double>'.format(low_val)+\
        '\n						</right>'+\
        '\n					</comparison-function>'+\
        '\n				</evaluation-tree>'

    xml+='\n			</mapping-function-2-value>'+\
        '\n			<mapping-function-2-value order="1" value="Normal Low">'+\
        '\n				<evaluation-tree data-type="logical-function">'+\
        '\n					<logical-function logical-operator="and">'+\
        '\n						<operands>'+\
        '\n							<operand data-type="comparison-function">'+\
        '\n								<comparison-function comparison-operator="bigger-equal">'+\
        '\n									<left data-type="concept-id-allowed-values">'+\
        '\n										<concept-id-allowed-values id="{}"/>'.format(str(derived_id))+\
        '\n									</left>'+\
        '\n									<right data-type="double">'+\
        '\n										<double>{}</double>'.format(low_val)+\
        '\n									</right>'+\
        '\n								</comparison-function>'+\
        '\n							</operand>'+\
        '\n							<operand data-type="comparison-function">'+\
        '\n								<comparison-function comparison-operator="smaller-equal">'+\
        '\n									<left data-type="concept-id-allowed-values">'+\
        '\n										<concept-id-allowed-values id="{}"/>'.format(str(derived_id))+\
        '\n									</left>'+\
        '\n									<right data-type="double">'+\
        '\n										<double>{}</double>'.format(normal_low_val)+\
        '\n									</right>'+\
        '\n								</comparison-function>'+\
        '\n							</operand>'+\
        '\n						</operands>'+\
        '\n					</logical-function>'+\
        '\n				</evaluation-tree>'

    xml+='\n			</mapping-function-2-value>'+\
        '\n			<mapping-function-2-value order="2" value="Normal">'+\
        '\n				<evaluation-tree data-type="logical-function">'+\
        '\n					<logical-function logical-operator="and">'+\
        '\n						<operands>'+\
        '\n							<operand data-type="comparison-function">'+\
        '\n								<comparison-function comparison-operator="bigger-equal">'+\
        '\n									<left data-type="concept-id-allowed-values">'+\
        '\n										<concept-id-allowed-values id="{}"/>'.format(str(derived_id))+\
        '\n									</left>'+\
        '\n									<right data-type="double">'+\
        '\n										<double>{}</double>'.format(normal_low_val)+\
        '\n									</right>'+\
        '\n								</comparison-function>'+\
        '\n							</operand>'+\
        '\n							<operand data-type="comparison-function">'+\
        '\n								<comparison-function comparison-operator="smaller-equal">'+\
        '\n									<left data-type="concept-id-allowed-values">'+\
        '\n										<concept-id-allowed-values id="{}"/>'.format(str(derived_id))+\
        '\n									</left>'+\
        '\n									<right data-type="double">'+\
        '\n										<double>{}</double>'.format(normal_high_val)+\
        '\n									</right>'+\
        '\n								</comparison-function>'+\
        '\n							</operand>'+\
        '\n						</operands>'+\
        '\n					</logical-function>'+\
        '\n				</evaluation-tree>'

    xml+='\n			</mapping-function-2-value>'+\
        '\n			<mapping-function-2-value order="3" value="Normal High">'+\
        '\n				<evaluation-tree data-type="logical-function">'+\
        '\n					<logical-function logical-operator="and">'+\
        '\n						<operands>'+\
        '\n							<operand data-type="comparison-function">'+\
        '\n								<comparison-function comparison-operator="bigger-equal">'+\
        '\n									<left data-type="concept-id-allowed-values">'+\
        '\n										<concept-id-allowed-values id="{}"/>'.format(str(derived_id))+\
        '\n									</left>'+\
        '\n									<right data-type="double">'+\
        '\n										<double>{}</double>'.format(normal_high_val)+\
        '\n									</right>'+\
        '\n								</comparison-function>'+\
        '\n							</operand>'+\
        '\n							<operand data-type="comparison-function">'+\
        '\n								<comparison-function comparison-operator="smaller-equal">'+\
        '\n									<left data-type="concept-id-allowed-values">'+\
        '\n										<concept-id-allowed-values id="{}"/>'.format(str(derived_id))+\
        '\n									</left>'+\
        '\n									<right data-type="double">'+\
        '\n										<double>{}</double>'.format(high_val)+\
        '\n									</right>'+\
        '\n								</comparison-function>'+\
        '\n							</operand>'+\
        '\n						</operands>'+\
        '\n					</logical-function>'+\
        '\n				</evaluation-tree>'


    xml+='\n			</mapping-function-2-value>'+\
        '\n			<mapping-function-2-value order="4" value="High">'+\
        '\n				<evaluation-tree data-type="comparison-function">'+\
        '\n					<comparison-function comparison-operator="bigger">'+\
        '\n						<left data-type="concept-id-allowed-values">'+\
        '\n							<concept-id-allowed-values id="{}"/>'.format(str(derived_id))+\
        '\n						</left>'+\
        '\n						<right data-type="double">'+\
        '\n							<double>{}</double>'.format(high_val)+\
        '\n						</right>'+\
        '\n					</comparison-function>'+\
        '\n				</evaluation-tree>'+\
        '\n			</mapping-function-2-value>'+\
        '\n		</mapping-functions-to-values>'+\
        '\n	</mapping-function>'+\
        '\n	<ordinal-allowed-values output-type="ordinal">'

    xml+="\n		<persistence>"+\
        "\n			<global-persistence behavior=\"pos-pos\" granularity=\"second\">"+\
        "\n				<interpolation-table>"+\
        "\n					<rows/>"+\
        "\n				</interpolation-table>"+\
        "\n			</global-persistence>"+\
        "\n			<local-persistence>"+\
        "\n				<good-before value=\""+str(goodBeforeValue)+"\" granularity=\""+goodBeforeGranularity+"\"/>"+\
        "\n				<good-after value=\""+str(goodAfterValue)+"\" granularity=\""+goodAfterGranularity+"\"/>"+\
        "\n			</local-persistence>"+\
        "\n		</persistence>"

    xml+="\n		<values>"+\
        '\n			<ordinal-allowed-value order="0" value="Low"/>'+\
        '\n			<ordinal-allowed-value order="1" value="Normal Low"/>'+\
        '\n			<ordinal-allowed-value order="2" value="Normal"/>'+\
        '\n			<ordinal-allowed-value order="3" value="Normal High"/>'+\
        '\n			<ordinal-allowed-value order="4" value="High"/>'+\
        "\n		</values>"+\
        "\n	</ordinal-allowed-values>"+\
        "\n</state>"
    return name,xml


In [ ]:
C_T_ID=1;  C_T_NAME=2; C_T_DESC=3; C_T_TYPE=4; C_T_GOODBEFORE_VAL=11;  C_T_GOODBEFORE_GRAN=C_T_GOODBEFORE_VAL+1;
C_T_GOODAFTER_VAL=13 ; C_T_GOODAFTER_GRAN=C_GOODAFTER_VAL+1; C_T_SIG_VAR = 15; C_T_T_STEADY = 16
C_T_WRAPPINGID = 17

In [ ]:
def generateXML_trend(row):
    xml="<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n"
    xml+='<';
    id=int(get(row,C_T_ID));
    derived_id =int(get(row,C_T_WRAPPINGID))
    name=get(row,C_T_NAME); desc=get(row,C_T_DESC);  type='trend'
    goodBeforeValue=get(row,C_T_GOODBEFORE_VAL);    goodBeforeGranularity=get(row,C_T_GOODBEFORE_GRAN)
    goodAfterValue=get(row,C_T_GOODAFTER_VAL) ;     goodAfterGranularity=get(row,C_T_GOODAFTER_GRAN)
    significant_var = get(row,C_T_SIG_VAR);         t_steady = get(row,C_T_T_STEADY)

    if desc is None:
        desc = ''
    xml+='trend id=\"'+str(id)+"\" "+"name=\""+name+'_TREND'"\" concept-type=\""+\
    type+"\" description=\""+desc+"\" significant-variation=\""+str(significant_var)+\
    '" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:noNamespaceSchemaLocation=\"C:\\Users\\mashi\\data science\\thesis\\16.10.xsd\"> '+\
    "\n	<categories/>"+\
    "\n	<standard-terms/>"+\
    "\n	<synonyms/>"+\
    "\n	<temporal-semantic downward-hereditary=\"true\" forward=\"true\" backward=\"true\" solid=\"true\" concatenable=\"true\" gestalt=\"true\"/>"+\
    "\n	<clippers/>"+\
    "\n	<derived-from>"+\
    "\n		<derived-from-id>"+str(derived_id)+"</derived-from-id>"+\
    "\n	</derived-from>"+\
    "\n	<abstraction-at-contexts/>"+\
    '\n	<gradient-trend-allowed-values output-type="ordinal">'+\
    "\n		<persistence>"+\
    "\n			<global-persistence behavior=\"pos-pos\" granularity=\"second\">"+\
    '\n				<interpolation-table>'+\
    '\n					<rows />'+\
    '\n				</interpolation-table>'+\
    '\n			</global-persistence>'+\
    '\n			<local-persistence>'+\
    "\n				<good-before value=\""+str(goodBeforeValue)+"\" granularity=\""+goodBeforeGranularity+"\"/>"+\
    "\n				<good-after value=\""+str(goodAfterValue)+"\" granularity=\""+goodAfterGranularity+"\"/>"+\
    "\n			</local-persistence>"+\
    "\n		</persistence>"+\
    "\n		<values>"+\
    '\n			<ordinal-allowed-value order="0" value="DEC" />'+\
    '\n			<ordinal-allowed-value order="1" value="SAME" />'+\
    '\n			<ordinal-allowed-value order="2" value="INC" />'+\
    "\n		</values>"+\
    "\n		</gradient-trend-allowed-values>"+\
    '\n		<time-steady value="{}" granularity="hour" />'.format(str(t_steady))+\
    '\n</trend>'
    return name,xml


In [ ]:

path='\\content\\'
loc_raw = 'Diabetes_test_numeric_TAKgen_RAW.xlsx'

# wb = xlrd.open_workbook(loc)
# sheet = wb.sheet_by_index(0)
wb = openpyxl.load_workbook(loc_raw)
sheet = wb.active  # Get the active sheet

for row in range(4,5):
    name,xml = generateXML_raw_concept(row)
    if  ":" in name :
        name = name.replace(":", "_")
    if  "/" in name :
        name = name.replace("/", "_")
    print("Generating XML for concept :<"+name+">")
    f= open(path+'TAKgenXMLs\\raw_concepts\\'+name+"_RAW.xml","w+")
    f.write(xml)
    f.close()

wb.close()

#     wrappedID = get(row,C_WRAPPINGID)
#     if  (wrappedID != None):
#         name,xml = generateWrappingStateXML(row)
#         print("Generating (wrapping)XML for concept :<"+name+">")
#         f= open(path+'TAKgenXMLs\\raw_concepts\\'+name+".xml","w+")
#         f.write(xml)
#         f.close()
#     #print(xml)
#     #print("------------------------------------")


Generating XML for concept :<Glucose>


In [ ]:
# path='C:\\Users\User\Desktop\\'
# loc_state = path+'Diabetes_test_numeric_TAKgen_RAW.xlsx'

loc_state = 'Diabetes_test_numeric_TAKgen_RAW.xlsx'


# wb = xlrd.open_workbook(loc)
# sheet = wb.sheet_by_index(0)
wb = openpyxl.load_workbook(loc_state)
sheet = wb.active  # Get the active sheet

for row in range(4,5):
    name,xml = generateXML_state(row)
    if  ":" in name:
        name = name.replace(":", "_")
    if  "/" in name :
        name = name.replace("/", "_")
    print("Generating XML for concept :<"+name+">")
    f= open(path+'TAKgenXMLs\\state_concepts\\'+name+"_STATE.xml","w+")
    f.write(xml)
    f.close()
wb.close()

Generating XML for concept :<Glucose>


In [ ]:
path='C:\\research\\'
loc_raw = path+'blood_test_numeric_TAKgen_TREND.xlsx'

# wb = xlrd.open_workbook(loc)
# sheet = wb.sheet_by_index(0)
wb = openpyxl.load_workbook(loc_raw)
sheet = wb.active  # Get the active sheet

for row in range(16,44):
    name,xml = generateXML_trend(row)
    if  ":" in name :
        name = name.replace(":", "_")
    if  "/" in name :
        name = name.replace("/", "_")
    print("Generating XML for concept :<"+name+">")
    f= open(path+'TAKgenXMLs\\trend_concepts\\'+name+"_TREND.xml","w+")
    f.write(xml)
    f.close()

wb.close()

Generating XML for concept :<HCT>
Generating XML for concept :<ESR>
Generating XML for concept :<BASOPHILES>
Generating XML for concept :<BASOPHILES%>
Generating XML for concept :<EOSINOPHILS>
Generating XML for concept :<EOSINOPHILS%>
Generating XML for concept :<HGB>
Generating XML for concept :<LYMPHOCYTE>
Generating XML for concept :<LYMPHOCYTE%>
Generating XML for concept :<MCH>
Generating XML for concept :<MCHC>
Generating XML for concept :<MCV>
Generating XML for concept :<MONOCYTE>
Generating XML for concept :<MONOCYTE%>
Generating XML for concept :<MPV>
Generating XML for concept :<NEUTROPHILE>
Generating XML for concept :<NEUTROPHILE%>
Generating XML for concept :<PLATELETS>
Generating XML for concept :<RBC>
Generating XML for concept :<RDW>
Generating XML for concept :<WBC>
Generating XML for concept :<INR>
Generating XML for concept :<PT%>
Generating XML for concept :<PTSEC>
Generating XML for concept :<PTT>
Generating XML for concept :<RETICULOCYTES>
Generating XML for con

In [ ]:
def generateWrappingStateXML(row):
    persistenceXml=""
    stateId=int(get(row,C_WRAPPINGID))
    id=int(get(row,C_ID))
    allowedValuesList=get(row,C_NOM_ALLOWEDVALS)
    try :
        allowedValuesList=allowedValuesList.split(',')
        strippedTemp =[]
        for val in allowedValuesList:
            val=val.strip()
            strippedTemp.append(val)
        allowedValuesList=strippedTemp
    except:
        pass

    name=get(row,C_NAME)+".state"
    desc=get(row,C_DESC)
    type=get(row,C_TYPE)
    xml="<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n"
    xml+="<state id=\""+str(stateId)+"\" name=\"" +name+ "\" concept-type=\"state\" description=\""+name+"\" semi-formal=\"String\" "+\
         "source=\"String\" xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:noNamespaceSchemaLocation=\"C:\\Users\\mashi\\data science\\thesis\\16.10.xsd\">"+\
	     "\n<categories/>"+\
	     "\n<standard-terms/>"+\
	     "\n<synonyms/>"+\
         "\n<temporal-semantic downward-hereditary=\"true\" forward=\"true\" backward=\"true\" solid=\"true\" concatenable=\"true\" gestalt=\"true\"/>"+\
	     "\n<clippers/>"+\
	     "\n<derived-from>"+\
         "\n<derived-from-id>"+str(id)+"</derived-from-id>"+\
	     "\n</derived-from>"+\
	     "\n<abstraction-at-contexts/>"

    persistenceXml="\n<persistence>\n<global-persistence granularity=\"second\">"+\
  			"\n<interpolation-table>\n<rows/>\n</interpolation-table>\n</global-persistence>\n<local-persistence>"+\
  			"\n<good-before value=\"\" granularity=\"second\"/>\n<good-after value=\"\" granularity=\"second\"/>"+\
  		"\n</local-persistence>\n</persistence>"

    if (type=='raw-nominal'):
        xml+="\n<mapping-function>\n<mapping-functions-to-values>"
        xmlMappingFunction="";
        i=0
        for value in allowedValuesList:
            xmlMappingFunction += "\n<mapping-function-2-value order=\""+str(i)+"\" value=\""+value+"\">" +\
	    			                "\n<evaluation-tree data-type=\"comparison-function\">\n<comparison-function comparison-operator=\"equal\">"+\
	    					        "\n<left data-type=\"concept-id-allowed-values\">\n<concept-id-allowed-values id=\""+str(id)+"\"/>"+\
	    					        "\n</left>\n<right data-type=\"string\">\n<string>"+value+"</string>\n</right>"+\
	    				            "\n</comparison-function>\n</evaluation-tree>\n</mapping-function-2-value>"
            i+=1
        xml+= xmlMappingFunction
        xml+= "\n</mapping-functions-to-values>\n</mapping-function>\n<nominal-allowed-values output-type=\"nominal\">"
        xml+= persistenceXml
        xml+="\n<values>"
        for value in allowedValuesList: xml+="\n<nominal-allowed-value value=\""+value+"\"/>"
        xml+= "\n</values>\n</nominal-allowed-values>\n</state>"

    if (type =='raw-numeric'):
        minVal=int(get(row,C_NUM_MIN))
        maxVal=int(get(row,C_NUM_MAX))
        xml+="<mathematical-function mathematical-operator=\"plus\">\n"+\
         "<left data-type=\"double\">\n<concept-id-allowed-values id=\""+str(id)+"\" />\n</left>"+\
             "<right data-type=\"double\">\n<double>0</double>\n</right>\n</mathematical-function>"
        xml+="<numeric-allowed-values output-type=\"numeric\" min-value=\""+ str(minVal) +\
              "\" max-value=\""+ str(maxVal) +"\" units=\"\" scale=\"ratio\">"
        xml+=persistenceXml
        xml+= "</numeric-allowed-values>\n</state>"

    return name,xml
